# PUMA growth graphic

For RHNA option.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gp
from census import Census
import os
from glob import glob
import matplotlib.patches as mpatch

In [ ]:
plt.style.use('asu-light')

In [ ]:
capi = Census(os.environ['CENSUS_API_KEY'])

In [ ]:
pumas = gp.read_file('/Volumes/Pheasant Ridge/IPUMS/pumas/socal_pumas_projected.shp').dissolve('PUMA')

In [ ]:
# plot out just the rent new MF
land = gp.read_file('../../sorting/data/ne_10m_land.shp').to_crs(epsg=26911)

roads = pd.concat([gp.read_file(i).to_crs(epsg=26911) for i in glob('../../sorting/data/tl_roads/*.shp')], ignore_index=True)

counties = gp.read_file('../../sorting/data/counties/tl_2019_us_county.shp').to_crs(26911)
counties = counties[(counties.STATEFP == '06') & counties.NAME.isin(['Los Angeles', 'Ventura', 'Orange', 'Riverside', 'San Bernardino', 'Imperial'])]

In [ ]:
growth_rates = pd.read_csv('../data/puma_growth.csv')

In [ ]:
growth_rates.head()

In [ ]:
growth_rates[['puma', 'mf']] = growth_rates['Unnamed: 0'].str.split('_', expand=True)

In [ ]:
pumas

In [ ]:
colors = {
    (0, 0): ('0%', '#f8f8f8'),
    (0, 1): ('< 1%', '#b2e2e2'),
    (1, 5): ('1–5%', '#66c2a4'),
    (5, 25): ('5–25%', '#2ca25f'),
    (25, np.inf): ('≥25%', '#006d2c')
}

def color_for_val (val):
    if pd.isnull(val):
        return '#ffffff'
    
    for rnge, spec in colors.items():
        if rnge[0] == rnge[1] and val == rnge[0]:
            return spec[1]
        elif val >= rnge[0] and val < rnge[1]:
            return spec[1]
    else:
        raise ValueError(f'Value {val} not in any range!')

def map_const (rates, ax=None, draw_map=True, draw_hqta=False, legend=True, inset=True, _inset=False):
    if ax is None:
        f, ax = plt.subplots(figsize=(10, 10))
        
    if draw_map:
        puma_development = pumas.merge(rates, left_index=True, right_on='puma', validate='m:1')
        puma_development.to_crs(epsg=26911).plot(ax=ax, color=(puma_development.total_growth  * 100).apply(color_for_val))
        roads.plot(color='#888888', ax=ax, lw=0.25)
        counties.plot(edgecolor='#000',  facecolor='none', ax=ax, lw=1)
        
#         if draw_hqta:
#             hqta.plot(ax=ax, color='#00a3e0', alpha=0.3, lw=1)
        
        #water.plot(color='#aaaaaa', ax=ax)
        if _inset:
            ax.set_ylim(3.73e6, 3.79e6)
            ax.set_xlim(3.1e5, 4.2e5)
        elif draw_hqta:
            ax.set_ylim(3.67e6, 3.83e6)
            ax.set_xlim(3.45e5, 5.8e5)
        else:
            ax.set_ylim(3.59e6, 3.98e6)
            ax.set_xlim(2.75e5, 7.7e5)

        if inset and not _inset:
            inset_ax = ax.inset_axes([0.5, 0.61, 0.5, 0.4])
            inset_ax.set_xlabel('Central Los Angeles')
            map_const(rates, draw_hqta=draw_hqta, ax=inset_ax, legend=False, _inset=True)
            
    ax.set_xticks([])
    ax.set_yticks([])

    ax.set_yticks([])
    ax.set_xticks([])

    ax.set_aspect('equal')

    if legend:
        patches = [mpatch.Patch(color=c[1]) for c in colors.values()]
        labels = [c[0] for c in colors.values()]
        
#         if draw_hqta:
#             patches.append(mpatch.Patch(color='#00a3e0', alpha=0.3))
#             labels.append('High-quality transit area')
        
        ax.legend(
            patches,
            labels,
            loc='lower left' if draw_map else 'center',
            title='Housing unit growth',
            framealpha=1,
            fontsize='medium' if draw_map else 'large',
            title_fontsize='medium' if draw_map else 'large'
        )
        
        if not draw_map:
            ax.set_axis_off()
 

In [ ]:
f, axs = plt.subplots(2, 1, figsize=(6, 10))

for lbl, sfmf, ax in zip(('Single-family', 'Multi-family'), ('sf', 'mf'), axs):
    map_const(growth_rates[growth_rates.mf == sfmf], ax=ax, legend=sfmf=='sf')
    ax.set_title(lbl)

plt.savefig('../../dissertation/fig/sorting/rhna_growth.png', dpi=300, bbox_inches='tight')